<a href="https://colab.research.google.com/github/sandroklippel/misc_noteboooks/blob/main/Historico_Mapbiomas_de_um_arquivo_GeoJSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Histórico de Uso e Cobertura do Solo segundo Mapbiomas para um determinado polígono (`.geojson`)

Instruções:

1. Atribua o nome do projeto registrado no Google Earth Engine na variável `PROJECT`. Você encontra o nome do projeto em code.earthengine.google.com como no exemplo abaixo: 
![image.png](image.png)
2. Gere um arquivo `map.geojson` com a área desejada (polígono no SRC EPSG:4326). Pode ser criado no QGIS, ArcGIS ou no site https://geojson.io/.
3. Carregue esse arquivo no seu Google Drive. Se já houver um arquivo anterior, sobrescrever.
4. Selecione no menu "Ambiente de execução" -> "Executar tudo" ou pressione Ctrl+F9
5. Na primeira execução será necessário autorizar a conexão com o Google Drive. Siga as instruções.
6. O gráfico será gerado na última célula. Há opção para salvá-lo em .PNG.



In [ ]:
PROJECT=''

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import json
from google.colab import drive

drive.mount('/gdrive', force_remount=True)

with open('/gdrive/MyDrive/map.geojson') as f:
  feature = json.load(f)

for i in range(len(feature['features'])):
  feature['features'][i]['id'] = str(i) 


In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project=PROJECT)

In [ ]:
import datetime
import pandas as pd
import numpy as np
import plotly.express as px

from shapely.geometry import shape

def map_function(s):
  def mbyr(y):
      k = ee.String('classification_').cat(ee.String(y))
      d = ee.Dictionary(s.get(k))
      return d.set('year', y)
  return mbyr

classes = {1: 'Floresta',
           3: 'Formação Florestal',
           4: 'Formação Savânica',
           5: 'Mangue',
           6: 'Floresta Alagável (beta)',
           49: 'Restinga Arbórea',
           10: 'Formação Natural não Florestal',
           11: 'Campo Alagado e Área Pantanosa',
           12: 'Formação Campestre',
           32: 'Apicum',
           29: 'Afloramento Rochoso',
           50: 'Restinga Herbácea',
           13: 'Outras Formações não Florestais',
           14: 'Agropecuária',
           15: 'Pastagem',
           18: 'Agricultura',
           19: 'Lavoura Temporária',
           39: 'Soja',
           20: 'Cana',
           40: 'Arroz',
           62: 'Algodão (beta)',
           41: 'Outras Lavouras Temporárias',
           36: 'Lavoura Perene',
           46: 'Café',
           47: 'Citrus',
           35: 'Dendê (beta)',
           48: 'Outras Lavouras Perenes',
           9: 'Silvicultura',
           21: 'Mosaico de Usos',
           22: 'Área não Vegetada',
           23: 'Praia, Duna e Areal',
           24: 'Área Urbanizada',
           30: 'Mineração',
           25: 'Outras Áreas não Vegetadas',
           26: "Corpo D'água",
           33: 'Rio, Lago e Oceano',
           31: 'Aquicultura',
           27: 'Não observado'}

cores = {1: '#32a65e',
         3: '#1f8d49',
         4: '#7dc975',
         5: '#04381d',
         6: '#026975',
         49: '#02d659',
         10: '#ad975a',
         11: '#519799',
         12: '#d6bc74',
         32: '#fc8114',
         29: '#ffaa5f',
         50: '#ad5100',
         13: '#d89f5c',
         14: '#FFFFB2',
         15: '#edde8e',
         18: '#E974ED',
         19: '#C27BA0',
         39: '#f5b3c8',
         20: '#db7093',
         40: '#c71585',
         62: '#ff69b4',
         41: '#f54ca9',
         36: '#d082de',
         46: '#d68fe2',
         47: '#9932cc',
         35: '#9065d0',
         48: '#e6ccff',
         9: '#7a5900',
         21: '#ffefc3',
         22: '#d4271e',
         23: '#ffa07a',
         24: '#d4271e',
         30: '#9c0027',
         25: '#db4d4f',
         26: '#0000FF',
         33: '#2532e4',
         31: '#091077',
         27: '#ffffff'}

color_map = {classes[k]: v for k, v in cores.items()}

hoje = datetime.datetime.today().strftime('%d/%m/%Y')

geom = ee.FeatureCollection(feature).geometry()
area_ha = geom.area().getInfo() / 10000.0

mapbiomas = ee.Image('projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1')

stats = (mapbiomas.reduceRegion(
         reducer = ee.Reducer.frequencyHistogram(),
        geometry = geom,
        scale = 30,
        crs = 'EPSG:4326'
        ))

years = ee.List([str(i) for i in range(1985,2023)])

map_by_year = map_function(stats)

lst = years.map(map_by_year)

df = pd.DataFrame(lst.getInfo())
df.set_index('year', inplace=True)
df_perc = df.div(df.iloc[0,:].sum(), axis=1).mul(100).round(2)
df_perc.columns = [classes[int(c)] for c in df_perc.columns.tolist()]

In [ ]:
fig = px.bar(df_perc, barmode='stack', 
             color_discrete_map=color_map, 
             labels={"value": "% da área do polígono", "variable": "", "year": ""},
             title=f"Histórico de Uso e Cobertura do Solo ({area_ha:.2f} ha)")

fig.update_layout(template='simple_white')
fig.update_xaxes(tickangle=-90)
fig.add_annotation(
    text = f"Fonte dos dados: Projeto MapBiomas – Coleção 8 da Série Anual de Mapas de Uso e Cobertura da Terra do Brasil, acesso em {hoje}.",
    showarrow=False,
    x = 0,
    y = -0.2,
    xref='paper',
    yref='paper',
    xanchor='left',
    yanchor='bottom',
    xshift=-1,
    yshift=-5,
    font=dict(size=10, color="grey"),
    align="left")
fig.show()

### Polígono para simples conferência:

In [ ]:
shape(geom.getInfo())